In [ ]:
import os, glob, json, csv, subprocess, sys, re, operator
from git import *
from subprocess import Popen, PIPE
from os import path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objs as go
import plotly.plotly as ply
import plotly
from plotly.offline import plot, init_notebook_mode, iplot

%matplotlib inline

# Defining repository

In [ ]:
userhome = os.path.expanduser('~')
repository = os.path.dirname(userhome + r'/different-diff/dataset/hbase/')

In [ ]:
#set up the analyze directory
analyze_dir = userhome + r'/different-diff/analyze/analyze_hbase/'

# Defining function to execute git command

In [ ]:
def execute_command(cmd, work_dir):
    """Executes a shell command in a subprocess, waiting until it has completed.
 
    :param cmd: Command to execute.
    :param work_dir: Working directory path.
    """
    pipe = subprocess.Popen(cmd, shell=True, cwd=work_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    (out, error) = pipe.communicate()
    return out, error
    pipe.wait()

In [ ]:
algorithms = ['histogram','minimal','myers','patience']

# Comparing files with bugs introducing based on the number of line changed

Finding files and their paths with different number of line changed

In [ ]:
colnames = ['bug_id','bugintro_commitID','parent_id','filename','#deletions-hist','#deletions-myers','#deletions-min','#deletions-pat']

In [ ]:
#Show the csv file with number of line changed between diff-algorithms
data = pd.read_csv(analyze_dir + 'file-diff/diffbugs_file.csv')
data = data[colnames]
data

In [ ]:
data.iloc[154][3]

In [ ]:
dfhist = pd.read_csv(analyze_dir + 'file-diff/diffhist-bugs.csv')
dfmin = pd.read_csv(analyze_dir + 'file-diff/diffmin-bugs.csv')
dfmyers = pd.read_csv(analyze_dir + 'file-diff/diffmyers-bugs.csv')
dfpat = pd.read_csv(analyze_dir + 'file-diff/diffpat-bugs.csv')

In [ ]:
cols = ['bug_id','bugintro_commitID','parent_id','#insertions','#deletions','filename','#line_changed']
dthist = dfhist[cols][dfhist[cols]['#deletions'] != 0]
dtmin = dfmin[cols][dfmin[cols]['#deletions'] != 0]
dtmyers = dfmyers[cols][dfmyers[cols]['#deletions'] != 0]
dtpat = dfpat[cols][dfpat[cols]['#deletions'] != 0]
dthist

In [ ]:
dthist.iloc[568][5]

In [ ]:
data_algorithm = [dthist, dtmin, dtmyers, dtpat]
for da, dt_alg in enumerate(data_algorithm):
    filename = []
    for jj in range(0,len(dt_alg)):
        if dt_alg.iloc[jj]['#deletions'] != 0:
            filename.append(((dt_alg.iloc[jj][5].split('/'))[-1:])[0] + "_"+ dt_alg.iloc[jj][1] + "-" + dt_alg.iloc[jj][2][:10] + "_" + dt_alg.iloc[jj][0])

    with open(analyze_dir + 'file-diff/filename/filename_'+ algorithms[da] + '.txt', mode='wt', encoding="utf-8") as myfile:
        myfile.write('\n'.join(filename))

print ("Filename_<algorithm-name>.txt has been created")

In [ ]:
len(filename)

In [ ]:
cols = ['bug_id','bugintro_commitID','parent_id']
dtparent = pd.read_csv(analyze_dir + "file-diff/parentid.csv")
dtparent = dtparent[cols]
dtparent

# Finding the changes in files

In [ ]:
len(dthist)

In [ ]:
fn_hist = (open(analyze_dir + 'file-diff/filename/filename_histogram.txt')).read().split('\n')
fn_min = (open(analyze_dir + 'file-diff/filename/filename_minimal.txt')).read().split('\n')
fn_myers = (open(analyze_dir + 'file-diff/filename/filename_myers.txt')).read().split('\n')
fn_pat = (open(analyze_dir + 'file-diff/filename/filename_patience.txt')).read().split('\n')

In [ ]:
f_names = [fn_hist, fn_min, fn_myers, fn_pat]

In [ ]:
dthist.iloc[0][2]

In [ ]:
files = []
for n, d_algo in enumerate(data_algorithm):
    f = []
    for aa in range(0,len(d_algo)):
        sys.stdout.write('\r%i: ' %(n+1) + ' Extracting data %i' %(aa+1))
        sys.stdout.flush()
        name = analyze_dir + "file-diff/diff-files/" + f_names[n][aa] + "_" + algorithms[n] + "changes.diff"
        diff_cmd = "git diff -w --ignore-blank-lines --diff-algorithm=" + algorithms[n] + " " + d_algo.iloc[aa][2] + " " + d_algo.iloc[aa][1] + " -- " + d_algo.iloc[aa][5] + " > " + name
        execute_command(diff_cmd, repository)
        if os.stat(name).st_size == 0:
            try:
                os.remove(name)
            except FileNotFoundError:
                pass
        else:
            f.append(f_names[n][aa] + "_" + algorithms[n] + "changes.diff")
    files.append(f)
print('\nExtraction is complete')

In [ ]:
len(files[0])

In [ ]:
file_to_compare = []
for x in range(0,len(files[0])):
    fc = []
    for n,file in enumerate(files):
        fc.append(file[x])
    file_to_compare.append(fc)
file_to_compare

In [ ]:
i = 1
for z in file_to_compare:
    print (i,z)
    i += 1

# Define function to find different line in files

In [ ]:
def get_clean_line(line):
    return [entry.strip().lower() for entry in line if entry]

In [ ]:
path = analyze_dir + "file-diff/"
path1 = path + "diff-files/"
path2 = path + "diff-line-of-files/"

In [ ]:
fn = []
for xx, ftc in enumerate(file_to_compare):
    sys.stdout.write('\rComparing file %i' %(xx+1))
    sys.stdout.flush()
    diff_of_diff = []
    with open(path1 + ftc[0], 'r') as first_file, open(path1 + ftc[1], 'r') as second_file, open(path1 + ftc[2], 'r') as third_file, open(path1 + ftc[3], 'r') as fourth_file:
        first_reader = csv.reader(first_file)
        second_reader = csv.reader(second_file)
        third_reader = csv.reader(third_file)
        fourth_reader = csv.reader(fourth_file)

        for first_line, second_line, third_line, fourth_line in zip(first_reader, second_reader, third_reader, fourth_reader):
            first_list, second_list, third_list, fourth_list = get_clean_line(first_line), get_clean_line(second_line), get_clean_line(third_line), get_clean_line(fourth_line)

            if (len(first_list) != len(second_list) or sorted(first_list) != sorted(second_list)) or (len(first_list) != len(third_list) or sorted(first_list) != sorted(third_list)) or (len(first_list) != len(fourth_list) or sorted(first_list) != sorted(fourth_list)) or (len(second_list) != len(third_list) or sorted(second_list) != sorted(third_list)) or (len(second_list) != len(fourth_list) or sorted(second_list) != sorted(fourth_list)) or (len(third_list) != len(fourth_list) or sorted(third_list) != sorted(fourth_list)):
                temp = str('{} :=: {} :=: {} :=: {} :=: {}'.format(
                    first_reader.line_num, first_list, second_list, third_list, fourth_list)).replace("['",'').replace("']",'').split(":=:")
                diff_of_diff.append(temp)
    
    fname = re.search(("[^~]*[-]\d*"), ftc[0])
    fname = fname.group(0)
    
    with open (path2 + "diff_of_diff_" + fname + ".csv", 'w') as csvfile:
        cols = ['line_no','diffmyers-' + fname,'diffhistogram-' + fname,'diffminimal-' + fname,'diffpatience-' + fname]
        writers = csv.writer(csvfile, delimiter=',')
        writers.writerow(cols)
        for line in diff_of_diff:
            writers.writerow(line)
    
    fn.append(path2 + "diff_of_diff_" + fname + ".csv")
    
print ('\nComparing file is complete')

In [ ]:
len(fn)

In [ ]:
df = pd.read_csv(fn[124])
df

# Finding the buggy line of files

In [ ]:
f_names[2][1]

In [ ]:
buggylinefiles = []
for n, d_algo in enumerate(data_algorithm):
    bf = []
    for aa in range(0,len(d_algo)):
        sys.stdout.write('\r%i: ' %(n+1) + ' Extracting data %i' %(aa+1))
        sys.stdout.flush()
        name = analyze_dir + "file-diff/buggylines/" + algorithms[n] + "/" + f_names[n][aa] + "_" + algorithms[n] + "buglines_" + str(aa+1) + ".diff"
        diff_cmd = "git diff -U0 -w --ignore-blank-lines --diff-algorithm=" + algorithms[n] + " " + d_algo.iloc[aa][2] + " " + d_algo.iloc[aa][1] + " -- " + d_algo.iloc[aa][5] + " | grep '^[-]' | grep -Ev '^(--- a/|\+\+\+ b/)' > " + name
        execute_command(diff_cmd, repository)
        zzz = [d_algo.iloc[aa][0], d_algo.iloc[aa][1], d_algo.iloc[aa][2], d_algo.iloc[aa][5], (f_names[n][aa] + "_" + algorithms[n] + "buglines_" + str(aa+1) + ".diff")]
        bf.append(zzz)
    bf = sorted(bf, key=operator.itemgetter(4,1,2,0))
    buggylinefiles.append(bf)
print('\nExtraction is complete')

In [ ]:
for a,b in enumerate(buggylinefiles):
    print (b[a])

In [ ]:
bug_file = []
for m, algrt in enumerate(algorithms):
    temp_file = []
    s = 0
    for f_name in glob.iglob(path + 'buggylines/' + algrt + '/*', recursive=True):
        sys.stdout.write("\r%i " %(m+1) + "Counting bugline for file %i" %(s+1))
        sys.stdout.flush()
        s += 1
        
        temp_file.append(f_name)
    temp_file.sort()
    bug_file.append(temp_file)
print ("\nReading files are complete")

In [ ]:
pat = re.search("(?:\w+[-]\w+[-]\w+|\w+[-]\w+|\w+)[.]\w+[_]|(?<=histogram\/)(\w+[_])(?<=[_])",bug_file[0][321])
pat = pat.group()[:-1]
pat

In [ ]:
buggylinefiles[2][321]

# Counting number of files and number of buggylines

In [ ]:
for n, algr in enumerate(algorithms):
    total = []
    for rr, f_name in enumerate(bug_file[n]):
        sys.stdout.write("\r%i " %(n+1) + "Counting bugline for file %i" %(rr+1))
        sys.stdout.flush()
        
        fn = open(f_name, 'r')
        fn = fn.read().split('\n')
        if "--- /dev" in fn[0]:
            num = str(len(fn)-2)
        else:
            num = str(len(fn)-1)
        
        pattern = re.search("(?:\w+[-]\w+[-]\w+|\w+[-]\w+|\w+)[.]\w+[_]|(?<=" + algr + "\/)(\w+[_])(?<=[_])", f_name)
        pattern = pattern.group()[:-1]
        
        current = [buggylinefiles[n][rr][0], buggylinefiles[n][rr][1], buggylinefiles[n][rr][2], buggylinefiles[n][rr][3], pattern, num]
        total.append(current)
    
    total = sorted(total, key=operator.itemgetter(4,1,2,0))
    
    with open(path + "number_of_files/" + algr + "_total_bugline.csv", 'w') as myfile:
        writers = csv.writer(myfile)
        fields = ['bug_id','commit_id','parent_id','filepath','filename','number_of_buggyline']
        writers.writerow(fields)
        for row in total:
            writers.writerow(row)
            
print ("\nCounting buggyline is finish")

In [ ]:
df_totalhist = pd.read_csv(analyze_dir + 'file-diff/number_of_files/histogram_total_bugline.csv')
df_totalmin = pd.read_csv(analyze_dir + 'file-diff/number_of_files/minimal_total_bugline.csv')
df_totalmyers = pd.read_csv(analyze_dir + 'file-diff/number_of_files/myers_total_bugline.csv')
df_totalpat = pd.read_csv(analyze_dir + 'file-diff/number_of_files/patience_total_bugline.csv')

In [ ]:
d_frame = [df_totalhist, df_totalmin, df_totalmyers, df_totalpat]
for d, dfr in enumerate(d_frame):
    dframe = (pd.to_numeric(dfr['number_of_buggyline'], errors='coerce')
           .groupby(dfr['filepath'])
           .sum()
           .to_frame()
           .add_prefix('total')
           .reset_index())

    col = ['filepath','totalnumber_of_buggyline']
    dframe = dframe.dropna(subset=['totalnumber_of_buggyline'])
    dframe = dframe.sort_values('totalnumber_of_buggyline', ascending=False)

    dframe[col].to_csv(analyze_dir + 'file-diff/number_of_files/totalbug_of_' + algorithms[d] + '_filechanged.csv')

In [ ]:
dfrhist = pd.read_csv(analyze_dir + 'file-diff/number_of_files/totalbug_of_histogram_filechanged.csv')
dfrmin = pd.read_csv(analyze_dir + 'file-diff/number_of_files/totalbug_of_minimal_filechanged.csv')
dfrmyers = pd.read_csv(analyze_dir + 'file-diff/number_of_files/totalbug_of_myers_filechanged.csv')
dfrpat = pd.read_csv(analyze_dir + 'file-diff/number_of_files/totalbug_of_patience_filechanged.csv')

In [ ]:
filehist = dfrhist['filepath'][dfrhist['totalnumber_of_buggyline'] != 0].count()
filemin = dfrmin['filepath'][dfrmin['totalnumber_of_buggyline'] != 0].count()
filemyers = dfrmyers['filepath'][dfrmyers['totalnumber_of_buggyline'] != 0].count()
filepat = dfrpat['filepath'][dfrpat['totalnumber_of_buggyline'] != 0].count()

bglinehist = dfrhist['totalnumber_of_buggyline'].sum()
bglinemin = dfrmin['totalnumber_of_buggyline'].sum()
bglinemyers = dfrmyers['totalnumber_of_buggyline'].sum()
bglinepat = dfrpat['totalnumber_of_buggyline'].sum()

print (bglinehist,bglinemin,bglinemyers,bglinepat)

In [ ]:
trace = go.Bar(
    x = algorithms,
    y = [filehist, filemin, filemyers, filepat],
    text = [filehist, filemin, filemyers, filepat],
    textposition = 'auto',
    opacity = 0.6,
    marker = dict(
        color = 'rgb(150,255,200)',
        line = dict(
            color='rgb(8,48,107)',
            width=1.5),
    )
)

dtbar = [trace]

layout = go.Layout(
    title = 'Number of files changed'
)

init_notebook_mode(connected=True)
fig = go.Figure(data=dtbar, layout=layout)
iplot(fig, show_link=False)

In [ ]:
graph = go.Bar(
    x = algorithms,
    y = [bglinehist, bglinemin, bglinemyers, bglinepat],
    text = [bglinehist, bglinemin, bglinemyers, bglinepat],
    textposition = 'auto',
    opacity = 0.6,
    marker = dict(
        color = 'rgb(150,255,200)',
        line = dict(
            color='rgb(8,48,107)',
            width=1.5),
    )
)

dtbar = [graph]

layout = go.Layout(
    title = 'Number of buglines'
)

init_notebook_mode(connected=True)
fig = go.Figure(data=dtbar, layout=layout)
iplot(fig, show_link=False)